In [1]:
import json
import cv2
import numpy as np

from torch.utils.data import Dataset

In [3]:
dataset_path = './training/fill50k/'
prompt_json = f"{dataset_path}prompt.json"

In [ ]:
# Sample prompt.json line
# {"source": "source/0.png", "target": "target/0.png", "prompt": "pale golden rod circle with old lace background"}


In [4]:

class MyDataset(Dataset):
    def __init__(self):
        self.data = []
        with open(prompt_json, 'rt') as f:
            for line in f:
                self.data.append(json.loads(line))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]

        source_filename = item['source']
        target_filename = item['target']
        prompt = item['prompt']

        source = cv2.imread(dataset_path + source_filename)
        target = cv2.imread(dataset_path + target_filename)

        # Do not forget that OpenCV read images in BGR order.
        source = cv2.cvtColor(source, cv2.COLOR_BGR2RGB)
        target = cv2.cvtColor(target, cv2.COLOR_BGR2RGB)

        # Normalize source images to [0, 1].
        source = source.astype(np.float32) / 255.0

        # Normalize target images to [-1, 1].
        target = (target.astype(np.float32) / 127.5) - 1.0

        return dict(jpg=target, txt=prompt, hint=source)


In [13]:
import  random 

dataset = MyDataset()
print(len(dataset))
sample_idx = random.randint(0, len(dataset)-1)
item = dataset[sample_idx]
jpg = item['jpg']
txt = item['txt']
hint = item['hint']
print(txt)
print(jpg.shape)
print(hint.shape)


50000
tan circle with pale green background
(512, 512, 3)
(512, 512, 3)


In [18]:
# Add Control
INPUT_MODEL = "./models/v1-5-pruned.ckpt"
OUTPUT_MODEL = "./models/control_sd15_ini.ckpt"

In [20]:
import os


assert os.path.exists(INPUT_MODEL), 'Input model does not exist.'
assert not os.path.exists(OUTPUT_MODEL), 'Output filename already exists.'
assert os.path.exists(os.path.dirname(OUTPUT_MODEL)), 'Output path is not valid.'

import torch
from share import *
from cldm.model import create_model


def get_node_name(name, parent_name):
    if len(name) <= len(parent_name):
        return False, ''
    p = name[:len(parent_name)]
    if p != parent_name:
        return False, ''
    return True, name[len(parent_name):]


model = create_model(config_path='./models/cldm_v15.yaml')

pretrained_weights = torch.load(INPUT_MODEL)
if 'state_dict' in pretrained_weights:
    pretrained_weights = pretrained_weights['state_dict']

scratch_dict = model.state_dict()

target_dict = {}
for k in scratch_dict.keys():
    is_control, name = get_node_name(k, 'control_')
    if is_control:
        copy_k = 'model.diffusion_' + name
    else:
        copy_k = k
    if copy_k in pretrained_weights:
        target_dict[k] = pretrained_weights[copy_k].clone()
    else:
        target_dict[k] = scratch_dict[k].clone()
        print(f'These weights are newly added: {k}')

model.load_state_dict(target_dict, strict=True)
torch.save(model.state_dict(), OUTPUT_MODEL)
print('Done.')


ControlLDM: Running in eps-prediction mode
DiffusionWrapper has 859.52 M params.
making attention of type 'vanilla' with 512 in_channels
Working with z of shape (1, 4, 32, 32) = 4096 dimensions.
making attention of type 'vanilla' with 512 in_channels
Loaded model config from [./models/cldm_v15.yaml]
These weights are newly added: logvar
These weights are newly added: control_model.zero_convs.0.0.weight
These weights are newly added: control_model.zero_convs.0.0.bias
These weights are newly added: control_model.zero_convs.1.0.weight
These weights are newly added: control_model.zero_convs.1.0.bias
These weights are newly added: control_model.zero_convs.2.0.weight
These weights are newly added: control_model.zero_convs.2.0.bias
These weights are newly added: control_model.zero_convs.3.0.weight
These weights are newly added: control_model.zero_convs.3.0.bias
These weights are newly added: control_model.zero_convs.4.0.weight
These weights are newly added: control_model.zero_convs.4.0.bias
T

In [24]:
import pytorch_lightning as pl
from torch.utils.data import DataLoader
from tutorial_dataset import MyDataset
from cldm.logger import ImageLogger
from cldm.model import create_model, load_state_dict


# Configs
resume_path = OUTPUT_MODEL
batch_size = 2
logger_freq = 300
learning_rate = 1e-5
sd_locked = True
only_mid_control = False

# accumulate_grad_batches=4

# First use cpu to load models. Pytorch Lightning will automatically move it to GPUs.
model = create_model('./models/cldm_v15.yaml').cpu()
model.load_state_dict(load_state_dict(resume_path, location='cpu'))
model.learning_rate = learning_rate
model.sd_locked = sd_locked
model.only_mid_control = only_mid_control


# Misc
dataset = MyDataset()
dataloader = DataLoader(dataset, num_workers=0, batch_size=batch_size, shuffle=True)
logger = ImageLogger(batch_frequency=logger_freq)
trainer = pl.Trainer(gpus=2, precision=32, callbacks=[logger], check_val_every_n_epoch=100)


# Train!
trainer.fit(model, dataloader)


ControlLDM: Running in eps-prediction mode
DiffusionWrapper has 859.52 M params.
making attention of type 'vanilla' with 512 in_channels
Working with z of shape (1, 4, 32, 32) = 4096 dimensions.
making attention of type 'vanilla' with 512 in_channels
Loaded model config from [./models/cldm_v15.yaml]
Loaded state_dict from [./models/control_sd15_ini.ckpt]


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


No module 'xformers'. Proceeding without it.


initializing distributed: GLOBAL_RANK: 0, MEMBER: 1/2


No module 'xformers'. Proceeding without it.


...


DDIM Sampler:  98%|█████████▊| 49/50 [00:19<00:00,  2.50it/s]

Epoch 0:  46%|████▌     | 5701/12500 [4:37:58<5:31:30,  2.93s/it, loss=0.00314, v_num=3, train/loss_simple_step=0.00237, train/loss_vlb_step=1.03e-5, train/loss_step=0.00237, global_step=5700.0]

DDIM Sampler: 100%|██████████| 50/50 [00:19<00:00,  2.52it/s]


Epoch 0:  46%|████▋     | 5800/12500 [4:42:44<5:26:36,  2.92s/it, loss=0.00462, v_num=3, train/loss_simple_step=0.00585, train/loss_vlb_step=2.37e-5, train/loss_step=0.00585, global_step=5799.0]  

: 